# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv('/Users/thiccboiii/python-api-challenge/WeatherPy/cities.csv')


# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,7.95,67,75,1.54,US,1715247741
1,1,okulovka,58.4008,33.2908,5.31,29,100,6.49,RU,1715247815
2,2,mamuju,-2.6748,118.8885,26.89,87,99,2.21,ID,1715247815
3,3,queenstown,-31.8976,26.8753,28.11,20,4,8.91,ZA,1715247815
4,4,iguape,-24.7081,-47.5553,22.63,66,12,3.48,BR,1715247815


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

required_columns = {'City', 'Lat', 'Lng', 'Humidity'}
if not required_columns.issubset(city_data_df.columns):
    raise ValueError(f"The DataFrame is missing one or more required columns: {required_columns}")

# Create a scatter plot using hvPlot
city_map = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    size='Humidity',       # Proportional marker size based on humidity
    color='blue',          # All points colored in blue
    hover_cols=['City', 'Humidity'],  # Hover to show additional information
    geo=True,              # Enable geographic mapping
    tiles='OSM',           # OpenStreetMap tiles
    xlabel='Longitude',
    ylabel='Latitude',
    title='City Humidity Map'
)
# Display the map
# YOUR CODE HERE


city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

# Define ideal weather criteria (adjust these as needed)
min_temp = 20  # Minimum temperature in degrees Celsius
max_temp = 30  # Maximum temperature in degrees Celsius
max_wind_speed = 5  # Maximum wind speed in meters per second
max_cloudiness = 20  # Maximum cloudiness in percentage

# Ensure necessary columns are present
required_columns = {'City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date'}
if not required_columns.issubset(city_data_df.columns):
    raise ValueError(f"The DataFrame is missing one or more required columns: {required_columns}")

# Drop any rows with null values
# YOUR CODE HERE

ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= min_temp) &
    (city_data_df['Max Temp'] <= max_temp) &
    (city_data_df['Wind Speed'] <= max_wind_speed) &
    (city_data_df['Cloudiness'] <= max_cloudiness)
].dropna()

# Display sample data
# YOUR CODE HERE

ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,iguape,-24.7081,-47.5553,22.63,66,12,3.48,BR,1715247815
24,24,edgewater,28.9889,-80.9023,24.51,86,0,4.17,US,1715247783
33,33,dowlatabad,32.7998,51.6955,20.82,16,20,3.09,IR,1715247819
42,42,kingston,17.9970,-76.7936,27.53,83,20,1.54,JM,1715247820
57,57,bikenibeu village,1.3673,173.1241,28.98,79,20,1.54,KI,1715247737


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE


hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

hotel_df['Hotel Name'] = ''

# Display sample data
# YOUR CODE HERE

hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
4,iguape,BR,-24.7081,-47.5553,66,
24,edgewater,US,28.9889,-80.9023,86,
33,dowlatabad,IR,32.7998,51.6955,16,
42,kingston,JM,17.9970,-76.7936,83,
57,bikenibeu village,KI,1.3673,173.1241,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
#geoAPI key 
geoapify_key = "77ca6f9acc794bf4bcbc7e85c36a6dd8"

# Set parameters to search for a hotel
radius = 10000
lat = ['Lat'] 
lng = ['Lng']

# Initialize the "Hotel Name" column if not done already
if "Hotel Name" not in hotel_df.columns:
    hotel_df["Hotel Name"] = ""


params = {
    "categories": "accommodation.hotel",
        "filter": f"circle:{lng},{lat},{radius}",
        "bias": f"proximity:{lng},{lat}",
        "limit": 1,
        "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = ['Lat'] 
    lng = ['Lng']


    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
iguape - nearest hotel: No hotel found
edgewater - nearest hotel: No hotel found
dowlatabad - nearest hotel: No hotel found
kingston - nearest hotel: No hotel found
bikenibeu village - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
liwale - nearest hotel: No hotel found
brezina - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
ha'il - nearest hotel: No hotel found
awantipur - nearest hotel: No hotel found
miracema do tocantins - nearest hotel: No hotel found
nuku'alofa - nearest hotel: No hotel found
terenos - nearest hotel: No hotel found
kishtwar - nearest hotel: No hotel found
abepura - nearest hotel: No hotel found
balykchy - nearest hotel: No hotel found
mermoz boabab - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
la'ie - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
fabens

,City,Country,Lat,Lng,Humidity,Hotel Name
4,iguape,BR,-24.7081,-47.5553,66,No hotel found
24,edgewater,US,28.9889,-80.9023,86,No hotel found
33,dowlatabad,IR,32.7998,51.6955,16,No hotel found
42,kingston,JM,17.9970,-76.7936,83,No hotel found
57,bikenibeu village,KI,1.3673,173.1241,79,No hotel found
61,port alfred,ZA,-33.5906,26.8910,86,No hotel found
66,liwale,TZ,-9.7667,37.9333,55,No hotel found
82,brezina,DZ,33.0989,1.2608,24,No hotel found
152,port elizabeth,ZA,-33.9180,25.5701,68,No hotel found
163,ha'il,SA,27.5219,41.6907,23,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

data = {
    "City": ["kapaa", "hilo", "banda", "makakilo city", "kahului"],
    "Country": ["US", "US", "IN", "US", "US"],
    "Lat": [22.0752, 19.7297, 25.4833, 21.3469, 20.8947],
    "Lng": [-159.3190, -155.0900, 80.3333, -158.0858, -156.4700],
    "Humidity": [84, 83, 52, 81, 60],
    "Hotel Name": ["Pono Kai Resort", "Dolphin Bay Hotel", "#acnindiafy21", "Embassy Suites by Hilton Oahu Kapolei", "Maui Seaside Hotel"]
}

# Create DataFrame
hotel_df = pd.DataFrame(data)


# Display the map
# YOUR CODE HERE

# Generate the map with custom hover information
city_map = hotel_df.hvplot.points(
    x='Lng', y='Lat',
    geo=True, tiles='OSM',
    color='City', size='Humidity',
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
    title="Cities with Hotel Information",
    frame_width=700, frame_height=500
)


# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)